In [9]:
import sqlite3
import requests
import csv
import pandas as pd

In [10]:
SCREENER = {
    "america": "United States", 
    "forex": "Forex", 
    # "crypto": "Cryptocurrency", 
    "indonesia": "Indonesia", 
    "india" : "India", 
    "cfd": "CFD", 
    "uk": "United Kingdom", 
    "brazil": "Brazil", 
    "vietnam": "Vietnam", 
    "rsa": "South Africa", 
    "ksa": "Saudi Arabia",
    "australia": "Australia", 
    "russia": "Russia", 
    "thailand": "Thailand", 
    "philippines": "Philippines", 
    "taiwan": "Taiwan", 
    "sweden": "Sweden", 
    "france": "France", 
    "turkey": "Turkey", 
    "euronext": "Euronext", 
    "germany": "Germany", 
    "spain": "Spain", 
    "hongkong": "Hong Kong", 
    "korea": "South Korea",
}


In [13]:
# Crete db if it doesn't exist
with sqlite3.connect('tradingview.db') as con:
    db = con.cursor()
    db.execute(
        "CREATE TABLE IF NOT EXISTS tv (screener TEXT, exchange TEXT, symbol TEXT, desc TEXT)")
    con.commit()

# Delete all rows
with sqlite3.connect('tradingview.db') as con:
    db = con.cursor()
    db.execute("DELETE FROM tv")
    con.commit()

In [19]:
data = []

for types in ["", "\"futures\""]:
    for x, _ in SCREENER.items():
        print(f"Loading screener: {x}")
        r = requests.post(f"https://scanner.tradingview.com/{x}/scan",
                          data=f'{{"symbols":{{"tickers":[],"query":{{"types":[{types}]}}}},"columns":["description"]}}')
        for res in r.json()["data"]:
            exchange, symbol = res["s"].split(":")
            desc = res["d"][0]
            # data.append({'screener':x,'exchange': exchange, 'symbol':symbol,'desc' : desc})
            data.append((x,exchange,symbol,desc))
            
        # print(data)
        # break
        # Use bulk operation for faster insert
        with sqlite3.connect('tradingview.db') as con:
            con.executemany(
                "INSERT INTO tv VALUES (?, ?, ?, ?)",
                data
            )
            con.commit()
        

Loading screener: america
Loading screener: forex
Loading screener: indonesia
Loading screener: india
Loading screener: cfd
Loading screener: uk
Loading screener: brazil
Loading screener: vietnam
Loading screener: rsa
Loading screener: ksa
Loading screener: australia
Loading screener: russia
Loading screener: thailand
Loading screener: philippines
Loading screener: taiwan
Loading screener: sweden
Loading screener: france
Loading screener: turkey
Loading screener: euronext
Loading screener: germany
Loading screener: spain
Loading screener: hongkong
Loading screener: korea
Loading screener: america
Loading screener: forex
Loading screener: indonesia
Loading screener: india
Loading screener: cfd
Loading screener: uk
Loading screener: brazil
Loading screener: vietnam
Loading screener: rsa
Loading screener: ksa
Loading screener: australia
Loading screener: russia
Loading screener: thailand
Loading screener: philippines
Loading screener: taiwan
Loading screener: sweden
Loading screener: fran

In [ ]:
# writing to csv

df = pd.DataFrame(data)
df.to_csv('Tickers.csv')